In [1]:
import pandas as pd

## Live Demo Code for Connecting to PostgreSQL from Python

In [2]:
!pip install psycopg2-binary

  Using cached psycopg2_binary-2.8.5-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)


In [3]:
import psycopg2

In [4]:
dir(psycopg2)

['BINARY',
 'Binary',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__libpq_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_connect',
 '_ext',
 '_ipaddress',
 '_json',
 '_psycopg',
 '_range',
 'apilevel',
 'compat',
 'connect',
 'errors',
 'extensions',
 'extras',
 'paramstyle',
 'threadsafety',
 'tz']

In [8]:
import dotenv
%cd ../

/home/joe/Desktop/school/3-2/module2-sql-for-analysis


In [9]:
help(psycopg2.connect)

Help on function connect in module psycopg2:

connect(dsn=None, connection_factory=None, cursor_factory=None, **kwargs)
    Create a new database connection.
    
    The connection parameters can be specified as a string:
    
        conn = psycopg2.connect("dbname=test user=postgres password=secret")
    
    or using a set of keyword arguments:
    
        conn = psycopg2.connect(database="test", user="postgres", password="secret")
    
    Or as a mix of both. The basic connection parameters are:
    
    - *dbname*: the database name
    - *database*: the database name (only as keyword argument)
    - *user*: user name used to authenticate
    - *password*: password used to authenticate
    - *host*: database host address (defaults to UNIX socket if not provided)
    - *port*: connection port number (defaults to 5432 if not provided)
    
    Using the *connection_factory* parameter a different class or connections
    factory can be specified. It should be a callable object tak

In [12]:
dotenv.load_dotenv()

db_host=os.getenv("DB_HOST")
db_pass=os.getenv("DB_SAFEWORD")
db_user=os.getenv("DB_USER")
db_port=os.getenv("DB_PORT")
db_table=os.getenv("DB_NAME")


In [13]:
conn = psycopg2.connect(dbname=db_table,
                        password=db_pass,
                        user=db_user,
                        port=db_port,
                        host=db_host)



In [14]:
conn

<connection object at 0x7ff630a34690; dsn: 'user=qewtsapf password=xxx dbname=qewtsapf host=drona.db.elephantsql.com port=5432', closed: 0>

In [15]:
# everytime you get something new its a good ide to take a look at the methods that are in it's scope
dir(conn)

['DataError',
 'DatabaseError',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'Warning',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'async',
 'async_',
 'autocommit',
 'binary_types',
 'cancel',
 'close',
 'closed',
 'commit',
 'cursor',
 'cursor_factory',
 'deferrable',
 'dsn',
 'encoding',
 'fileno',
 'get_backend_pid',
 'get_dsn_parameters',
 'get_native_connection',
 'get_parameter_status',
 'get_transaction_status',
 'info',
 'isexecuting',
 'isolation_level',
 'lobject',
 'notices',
 'notifies',
 'pgconn_ptr',
 'poll',
 'protocol_version',
 'readonly',
 'reset',
 'rollback',
 'server_version

In [16]:
curs=conn.cursor()

In [19]:
# create a new table
query="""
CREATE TABLE IF NOT EXISTS test_table 
(
  id SERIAL PRIMARY KEY,
  name varchar(40) NOT NULL,
  data JSONB
);
"""

# create a new table with the cursor object
curs.execute(query)
conn.commit()

In [20]:
# insert information into the new table
insert_query="""
INSERT INTO test_table (name, data) VALUES
('A row name', null),
('Another row, with JSON', '{ "a": 1, "b": ["dog", "cat", 42], "c": true }'::JSONB);
"""

curs.execute(insert_query)
conn.commit()

In [21]:
curs.execute("""SELECT * FROM test_table;""")
result = curs.fetchall()

In [22]:
result

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

# ETL - RPG data from SQLite to PostgreSQL

In [25]:
DATAPATH='data/rpg_db.sqlite3'

ls: cannot access 'DATAPATH': No such file or directory


In [27]:
import sqlite3
import pandas as pd;

In [28]:
sq_conn = sqlite3.connect(DATAPATH)
sq_curs = sq_conn.cursor()

In [29]:
sq_curs.execute("""SELECT COUNT(*) FROM charactercreator_character""")
result = sq_curs.fetchone()
result

(302,)

In [32]:
# our goal is to copy the character's table to apostgre using python
# step one EXtract all of the characters
get_characters = 'SELECT * FROM charactercreator_character'
sq_curs.execute(get_characters)
characters = sq_curs.fetchall()


In [34]:
characters[:5]

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1)]

In [35]:
# transform step
# in this case there is not that much to change in the underling data
# there isnt much getting changed its going for sql to sql

# but what do we need to load into postgre?
# going to need a new table in the destination
# create a new table
sq_curs.execute('PRAGMA table_info(charactercreator_character)').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [36]:
create_character_table="""
CREATE TABLE IF NOT EXISTS charactercreator_character (
    character_id SERIAL PRIMARY KEY,
    name varchar(30),
    level INT,
    exp INT,
    hp INT,
    strength INT,
    intelligence INT,
    dexterity INT,
    wisdom INT
);
"""
curs.execute(create_character_table)
conn.commit()

In [38]:
insert_query=f"""
INSERT INTO charactercreator_character
(name,level,exp,hp,strength,intelligence,dexterity,wisdom)
VALUES """ + str(characters[0][1:]) + ";"
print(insert_query)


INSERT INTO charactercreator_character
(name,level,exp,hp,stregth,intelligence,dexterity,wisdom)
VALUES ('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)


In [43]:
# how to do them for all characters
for character in characters:
    insert_query=f"""
    INSERT INTO charactercreator_character
    (name,level,exp,hp,strength,intelligence,dexterity,wisdom)
    VALUES """ + str(character[1:]) + ';'
    curs.execute(insert_query)
    #conn.commit()

In [44]:
conn.commit()

In [45]:
!ls

app  arron_s_lecture  data  README.md  sql


In [46]:
!ls data

rpg_db.sqlite3	titanic.csv


In [48]:
DATAPATH='data/titanic.csv'
df=pd.read_csv(DATAPATH)

In [49]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [56]:
create_character_table="""
CREATE TABLE IF NOT EXISTS titanic (
    index SERIAL PRIMARY KEY,
    survived varchar(30),
    pclass INT,
    name VARCHAR(50),
    sex VARCHAR(10),
    age REAL,
    siblings_Spouses_Aboard INT,
    Parents_Children_Aboard INT,
    Fair REAL
);
"""
curs.execute(create_character_table)
conn.commit()

In [58]:
help(df.to_sql)

Help on method to_sql in module pandas.core.generic:

to_sql(name: str, con, schema=None, if_exists: str = 'fail', index: bool = True, index_label=None, chunksize=None, dtype=None, method=None) -> None method of pandas.core.frame.DataFrame instance
    Write records stored in a DataFrame to a SQL database.
    
    Databases supported by SQLAlchemy [1]_ are supported. Tables can be
    newly created, appended to, or overwritten.
    
    Parameters
    ----------
    name : str
        Name of SQL table.
    con : sqlalchemy.engine.Engine or sqlite3.Connection
        Using SQLAlchemy makes it possible to use any DB supported by that
        library. Legacy support is provided for sqlite3.Connection objects. The user
        is responsible for engine disposal and connection closure for the SQLAlchemy
        connectable See `here                 <https://docs.sqlalchemy.org/en/13/core/connections.html>`_
    
    schema : str, optional
        Specify the schema (if database flavor sup

In [59]:
import csv
from io import StringIO
from sqlalchemy import create_engine

def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')
df.to_sql(f'{table_name}', engine, method=psql_insert_copy)

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': relation "sqlite_master" does not exist
LINE 1: SELECT name FROM sqlite_master WHERE type='table' AND name=?...
                         ^
